In [1]:
import orchest
import numpy as np
from pyclarify import *
from utils import Anomaly

In [2]:
data = orchest.get_inputs()

In [3]:
clarify_data = data['clarify_data']
energy_consumption_dfs = clarify_data['energy_consumption']

anomalies = data['anomalies_from_et']['anomalies_from_ett']

In [4]:
clarify_client = ClarifyClient("clarify-credentials.json")

In [5]:
building_name = list(energy_consumption_dfs['hourly'])[1]

energy_consumption_dfs['hourly'][building_name]

start = energy_consumption_dfs['hourly'][building_name].index[0]
end = energy_consumption_dfs['hourly'][building_name].index[-1]

num_hours = len(energy_consumption_dfs['hourly'][building_name].index)

In [6]:
anomalies[building_name].sort(key=lambda x: x.start)

# for a in anomalies[building_name]:
#     print(f'building: {a.building}    start: {a.start}    end: {a.end}')
    
print(len(anomalies[building_name]))

c = None
i = 0
while i < len(anomalies[building_name]):
    if anomalies[building_name][i].start == c:
         anomalies[building_name].pop(i)
    else:
        c = anomalies[building_name][i].start
        i += 1

print(len(anomalies[building_name]))

62
62


In [7]:
start

Timestamp('2019-01-01 00:00:00+0000', tz='UTC')

In [8]:
end

Timestamp('2020-01-01 23:00:00+0000', tz='UTC')

In [9]:
index_list = energy_consumption_dfs['hourly'][building_name].index

enum_list = np.zeros(num_hours, dtype=int)

for a in anomalies[building_name]:
    mask = np.logical_and(a.start <= index_list, index_list <= a.end)
    enum_list = enum_list + mask

print(enum_list)
print(np.count_nonzero(enum_list))
print(enum_list.shape[0] - np.count_nonzero(enum_list))

[1 1 1 ... 1 1 1]
1520
7262


In [10]:
type(enum_list[0])

numpy.int64

In [12]:
input_enum_id = 'test'
input_signal_id = 'sh'
enum_metadata = SignalInfo(name='testtest', description='noe', type='enum', enumValues={0: ' ', 1: 'anomaly'})

resp = clarify_client.save_signals(input_ids=[input_enum_id], signals=[enum_metadata])
t = [x.to_pydatetime() for x in index_list]
data = DataFrame(times=t, series={input_enum_id: [int(x) for x in enum_list]})

clarify_client.insert(data)



Response(jsonrpc='2.0', id='1', result=InsertResponse(signalsByInput={'test': InsertSummary(id='c97dd9jfgirmqa3oigq0', created=False)}), error=None)